In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.pipeline import Pipeline

In [3]:
wine_df=pd.read_csv("train.csv",index_col=0)
wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
Id,,,,,,,,,,,,
0,8.0,0.50,0.39,2.2,0.073,30.0,39.0,0.99572,3.33,0.77,12.1,6
1,9.3,0.30,0.73,2.3,0.092,30.0,67.0,0.99854,3.32,0.67,12.8,6
2,7.1,0.51,0.03,2.1,0.059,3.0,12.0,0.99660,3.52,0.73,11.3,7
3,8.1,0.87,0.22,2.6,0.084,11.0,65.0,0.99730,3.20,0.53,9.8,5
4,8.5,0.36,0.30,2.3,0.079,10.0,45.0,0.99444,3.20,1.36,9.5,6


In [4]:
X_train=wine_df.drop('quality',axis=1)
y_train=wine_df['quality']

In [7]:
test_df=pd.read_csv("test.csv",index_col=0)
test_df.sample()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
Id,,,,,,,,,,,
2336,7.0,0.5,0.22,2.0,0.081,21.0,67.0,0.99629,3.3,0.63,9.5


## 1. Std scaling + Linear

In [9]:
scaler=StandardScaler()

In [10]:
svm=SVC(kernel='linear',probability=True,random_state=23)

In [11]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)

In [14]:
pipe=Pipeline([('SCL',scaler),('SVM',svm)])

In [15]:
params={'SVM__C':np.linspace(0.001,5,5)}

In [16]:
gcv=GridSearchCV(pipe,param_grid=params,cv=kfold,scoring='neg_log_loss',verbose=3)
gcv.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END .....................SVM__C=0.001;, score=-1.094 total time=   0.8s
[CV 2/5] END .....................SVM__C=0.001;, score=-1.049 total time=   0.9s
[CV 3/5] END .....................SVM__C=0.001;, score=-1.058 total time=   0.8s
[CV 4/5] END .....................SVM__C=0.001;, score=-1.061 total time=   0.8s
[CV 5/5] END .....................SVM__C=0.001;, score=-1.058 total time=   0.8s
[CV 1/5] END ........SVM__C=1.2507499999999998;, score=-1.095 total time=   1.3s
[CV 2/5] END ........SVM__C=1.2507499999999998;, score=-1.034 total time=   1.2s
[CV 3/5] END ........SVM__C=1.2507499999999998;, score=-1.063 total time=   1.3s
[CV 4/5] END ........SVM__C=1.2507499999999998;, score=-1.051 total time=   1.3s
[CV 5/5] END ........SVM__C=1.2507499999999998;, score=-1.051 total time=   1.3s
[CV 1/5] END ........SVM__C=2.5004999999999997;, score=-1.101 total time=   1.7s
[CV 2/5] END ........SVM__C=2.5004999999999997;, 

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=Pipeline(steps=[('SCL', StandardScaler()),
                                       ('SVM',
                                        SVC(kernel='linear', probability=True,
                                            random_state=23))]),
             param_grid={'SVM__C': array([1.00000e-03, 1.25075e+00, 2.50050e+00, 3.75025e+00, 5.00000e+00])},
             scoring='neg_log_loss', verbose=3)

In [17]:
print(gcv.best_params_)
print(gcv.best_score_)

{'SVM__C': 5.0}
-1.057040775711506


In [18]:
best_model=gcv.best_estimator_

In [20]:
y_pred=best_model.predict(test_df)
y_pred

array([5, 5, 5, ..., 5, 5, 5])

In [23]:
sample_submission_df=pd.read_csv('sample_submission.csv',index_col=0)

In [24]:
sample_submission_df

,quality
Id,
2056,5
2057,5
2058,5
2059,5
2060,5
...,...
3423,5
3424,5
3425,5


In [25]:
sample_submission_df['quality']=y_pred

In [27]:
sample_submission_df.to_csv('sample_submission.csv')

Leaderboard Score
1. Score: 0.39382
2. Public score: 0.43906

## 2. MinMax scaling + Linear

In [30]:
scaler=MinMaxScaler()
svm=SVC(kernel='linear',probability=True,random_state=23)
pipe=Pipeline([('SCL',scaler),('SVM',svm)])
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)
params={'SVM__C':np.linspace(0.001,5,5)}

In [31]:
gcv=GridSearchCV(pipe,param_grid=params,cv=kfold,scoring='neg_log_loss',verbose=3)
gcv.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END .....................SVM__C=0.001;, score=-1.116 total time=   0.8s
[CV 2/5] END .....................SVM__C=0.001;, score=-1.060 total time=   0.8s
[CV 3/5] END .....................SVM__C=0.001;, score=-1.094 total time=   0.8s
[CV 4/5] END .....................SVM__C=0.001;, score=-1.082 total time=   0.8s
[CV 5/5] END .....................SVM__C=0.001;, score=-1.063 total time=   0.8s
[CV 1/5] END ........SVM__C=1.2507499999999998;, score=-1.104 total time=   0.7s
[CV 2/5] END ........SVM__C=1.2507499999999998;, score=-1.025 total time=   0.8s
[CV 3/5] END ........SVM__C=1.2507499999999998;, score=-1.064 total time=   0.8s
[CV 4/5] END ........SVM__C=1.2507499999999998;, score=-1.053 total time=   0.8s
[CV 5/5] END ........SVM__C=1.2507499999999998;, score=-1.055 total time=   0.8s
[CV 1/5] END ........SVM__C=2.5004999999999997;, score=-1.100 total time=   0.8s
[CV 2/5] END ........SVM__C=2.5004999999999997;, 

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=Pipeline(steps=[('SCL', MinMaxScaler()),
                                       ('SVM',
                                        SVC(kernel='linear', probability=True,
                                            random_state=23))]),
             param_grid={'SVM__C': array([1.00000e-03, 1.25075e+00, 2.50050e+00, 3.75025e+00, 5.00000e+00])},
             scoring='neg_log_loss', verbose=3)

In [32]:
print(gcv.best_params_)
print(gcv.best_score_)

{'SVM__C': 3.75025}
-1.0603063037946234


In [33]:
best_model=gcv.best_estimator_

In [35]:
y_pred=best_model.predict(test_df)
y_pred

array([5, 5, 5, ..., 5, 5, 5])

In [36]:
sample_submission_df['quality']=y_pred

In [37]:
sample_submission_df.to_csv('sample_submission.csv')

In [38]:
# Leaderboard Score
# Score: 0.39489
# Public score: 0.43906

## 3. Std scaling + Radial

In [40]:
scaler=StandardScaler()
svm=SVC(kernel='rbf',probability=True,random_state=23)
pipe=Pipeline([('SCL',scaler),('SVM',svm)])
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)

params={'SVM__C':np.linspace(0.001,5,10),'SVM__gamma':list(np.linspace(0.001,5,10))+['scale','auto']}

gcv=GridSearchCV(pipe,param_grid=params,cv=kfold,scoring='neg_log_loss',verbose=3)
gcv.fit(X_train,y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END ...SVM__C=0.001, SVM__gamma=0.001;, score=-1.174 total time=   1.1s
[CV 2/5] END ...SVM__C=0.001, SVM__gamma=0.001;, score=-1.162 total time=   1.1s
[CV 3/5] END ...SVM__C=0.001, SVM__gamma=0.001;, score=-1.125 total time=   1.1s
[CV 4/5] END ...SVM__C=0.001, SVM__gamma=0.001;, score=-1.177 total time=   1.1s
[CV 5/5] END ...SVM__C=0.001, SVM__gamma=0.001;, score=-1.249 total time=   1.1s
[CV 1/5] END SVM__C=0.001, SVM__gamma=0.5564444444444444;, score=-1.176 total time=   1.1s
[CV 2/5] END SVM__C=0.001, SVM__gamma=0.5564444444444444;, score=-1.106 total time=   1.1s
[CV 3/5] END SVM__C=0.001, SVM__gamma=0.5564444444444444;, score=-1.077 total time=   1.1s
[CV 4/5] END SVM__C=0.001, SVM__gamma=0.5564444444444444;, score=-1.085 total time=   1.1s
[CV 5/5] END SVM__C=0.001, SVM__gamma=0.5564444444444444;, score=-1.099 total time=   1.1s
[CV 1/5] END SVM__C=0.001, SVM__gamma=1.1118888888888887;, score=-1.176 total

[CV 5/5] END SVM__C=0.5564444444444444, SVM__gamma=2.778222222222222;, score=-1.199 total time=   2.3s
[CV 1/5] END SVM__C=0.5564444444444444, SVM__gamma=3.3336666666666663;, score=-1.225 total time=   2.3s
[CV 2/5] END SVM__C=0.5564444444444444, SVM__gamma=3.3336666666666663;, score=-1.204 total time=   2.3s
[CV 3/5] END SVM__C=0.5564444444444444, SVM__gamma=3.3336666666666663;, score=-1.211 total time=   2.3s
[CV 4/5] END SVM__C=0.5564444444444444, SVM__gamma=3.3336666666666663;, score=-1.206 total time=   2.3s
[CV 5/5] END SVM__C=0.5564444444444444, SVM__gamma=3.3336666666666663;, score=-1.214 total time=   2.3s
[CV 1/5] END SVM__C=0.5564444444444444, SVM__gamma=3.8891111111111107;, score=-1.232 total time=   2.3s
[CV 2/5] END SVM__C=0.5564444444444444, SVM__gamma=3.8891111111111107;, score=-1.216 total time=   2.3s
[CV 3/5] END SVM__C=0.5564444444444444, SVM__gamma=3.8891111111111107;, score=-1.220 total time=   2.3s
[CV 4/5] END SVM__C=0.5564444444444444, SVM__gamma=3.889111111111

[CV 3/5] END SVM__C=1.1118888888888887, SVM__gamma=scale;, score=-1.030 total time=   1.2s
[CV 4/5] END SVM__C=1.1118888888888887, SVM__gamma=scale;, score=-1.028 total time=   1.2s
[CV 5/5] END SVM__C=1.1118888888888887, SVM__gamma=scale;, score=-1.037 total time=   1.2s
[CV 1/5] END SVM__C=1.1118888888888887, SVM__gamma=auto;, score=-1.070 total time=   1.1s
[CV 2/5] END SVM__C=1.1118888888888887, SVM__gamma=auto;, score=-1.036 total time=   1.1s
[CV 3/5] END SVM__C=1.1118888888888887, SVM__gamma=auto;, score=-1.030 total time=   1.2s
[CV 4/5] END SVM__C=1.1118888888888887, SVM__gamma=auto;, score=-1.028 total time=   1.2s
[CV 5/5] END SVM__C=1.1118888888888887, SVM__gamma=auto;, score=-1.037 total time=   1.1s
[CV 1/5] END SVM__C=1.6673333333333331, SVM__gamma=0.001;, score=-1.078 total time=   1.1s
[CV 2/5] END SVM__C=1.6673333333333331, SVM__gamma=0.001;, score=-1.022 total time=   1.1s
[CV 3/5] END SVM__C=1.6673333333333331, SVM__gamma=0.001;, score=-1.055 total time=   1.1s
[CV 

[CV 2/5] END SVM__C=2.2227777777777775, SVM__gamma=1.6673333333333331;, score=-1.148 total time=   2.3s
[CV 3/5] END SVM__C=2.2227777777777775, SVM__gamma=1.6673333333333331;, score=-1.151 total time=   2.4s
[CV 4/5] END SVM__C=2.2227777777777775, SVM__gamma=1.6673333333333331;, score=-1.144 total time=   2.4s
[CV 5/5] END SVM__C=2.2227777777777775, SVM__gamma=1.6673333333333331;, score=-1.156 total time=   2.4s
[CV 1/5] END SVM__C=2.2227777777777775, SVM__gamma=2.2227777777777775;, score=-1.202 total time=   2.4s
[CV 2/5] END SVM__C=2.2227777777777775, SVM__gamma=2.2227777777777775;, score=-1.167 total time=   2.4s
[CV 3/5] END SVM__C=2.2227777777777775, SVM__gamma=2.2227777777777775;, score=-1.173 total time=   2.4s
[CV 4/5] END SVM__C=2.2227777777777775, SVM__gamma=2.2227777777777775;, score=-1.167 total time=   2.4s
[CV 5/5] END SVM__C=2.2227777777777775, SVM__gamma=2.2227777777777775;, score=-1.180 total time=   2.4s
[CV 1/5] END SVM__C=2.2227777777777775, SVM__gamma=2.77822222222

[CV 4/5] END SVM__C=2.778222222222222, SVM__gamma=3.8891111111111107;, score=-1.215 total time=   2.5s
[CV 5/5] END SVM__C=2.778222222222222, SVM__gamma=3.8891111111111107;, score=-1.222 total time=   2.5s
[CV 1/5] END SVM__C=2.778222222222222, SVM__gamma=4.444555555555556;, score=-1.235 total time=   2.5s
[CV 2/5] END SVM__C=2.778222222222222, SVM__gamma=4.444555555555556;, score=-1.221 total time=   2.5s
[CV 3/5] END SVM__C=2.778222222222222, SVM__gamma=4.444555555555556;, score=-1.224 total time=   2.5s
[CV 4/5] END SVM__C=2.778222222222222, SVM__gamma=4.444555555555556;, score=-1.222 total time=   2.5s
[CV 5/5] END SVM__C=2.778222222222222, SVM__gamma=4.444555555555556;, score=-1.227 total time=   2.4s
[CV 1/5] END SVM__C=2.778222222222222, SVM__gamma=5.0;, score=-1.238 total time=   2.5s
[CV 2/5] END SVM__C=2.778222222222222, SVM__gamma=5.0;, score=-1.224 total time=   2.4s
[CV 3/5] END SVM__C=2.778222222222222, SVM__gamma=5.0;, score=-1.226 total time=   2.5s
[CV 4/5] END SVM__C=

[CV 4/5] END SVM__C=3.8891111111111107, SVM__gamma=0.001;, score=-1.046 total time=   1.1s
[CV 5/5] END SVM__C=3.8891111111111107, SVM__gamma=0.001;, score=-1.046 total time=   1.1s
[CV 1/5] END SVM__C=3.8891111111111107, SVM__gamma=0.5564444444444444;, score=-1.132 total time=   2.1s
[CV 2/5] END SVM__C=3.8891111111111107, SVM__gamma=0.5564444444444444;, score=-1.099 total time=   2.0s
[CV 3/5] END SVM__C=3.8891111111111107, SVM__gamma=0.5564444444444444;, score=-1.106 total time=   2.1s
[CV 4/5] END SVM__C=3.8891111111111107, SVM__gamma=0.5564444444444444;, score=-1.095 total time=   2.0s
[CV 5/5] END SVM__C=3.8891111111111107, SVM__gamma=0.5564444444444444;, score=-1.122 total time=   2.1s
[CV 1/5] END SVM__C=3.8891111111111107, SVM__gamma=1.1118888888888887;, score=-1.160 total time=   2.3s
[CV 2/5] END SVM__C=3.8891111111111107, SVM__gamma=1.1118888888888887;, score=-1.124 total time=   2.3s
[CV 3/5] END SVM__C=3.8891111111111107, SVM__gamma=1.1118888888888887;, score=-1.130 total

[CV 2/5] END SVM__C=4.444555555555556, SVM__gamma=2.778222222222222;, score=-1.185 total time=   2.4s
[CV 3/5] END SVM__C=4.444555555555556, SVM__gamma=2.778222222222222;, score=-1.192 total time=   2.4s
[CV 4/5] END SVM__C=4.444555555555556, SVM__gamma=2.778222222222222;, score=-1.189 total time=   2.4s
[CV 5/5] END SVM__C=4.444555555555556, SVM__gamma=2.778222222222222;, score=-1.200 total time=   2.4s
[CV 1/5] END SVM__C=4.444555555555556, SVM__gamma=3.3336666666666663;, score=-1.223 total time=   2.5s
[CV 2/5] END SVM__C=4.444555555555556, SVM__gamma=3.3336666666666663;, score=-1.202 total time=   2.4s
[CV 3/5] END SVM__C=4.444555555555556, SVM__gamma=3.3336666666666663;, score=-1.208 total time=   2.4s
[CV 4/5] END SVM__C=4.444555555555556, SVM__gamma=3.3336666666666663;, score=-1.204 total time=   2.4s
[CV 5/5] END SVM__C=4.444555555555556, SVM__gamma=3.3336666666666663;, score=-1.213 total time=   2.5s
[CV 1/5] END SVM__C=4.444555555555556, SVM__gamma=3.8891111111111107;, score=

[CV 3/5] END ......SVM__C=5.0, SVM__gamma=auto;, score=-1.050 total time=   1.3s
[CV 4/5] END ......SVM__C=5.0, SVM__gamma=auto;, score=-1.049 total time=   1.3s
[CV 5/5] END ......SVM__C=5.0, SVM__gamma=auto;, score=-1.067 total time=   1.3s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=Pipeline(steps=[('SCL', StandardScaler()),
                                       ('SVM',
                                        SVC(probability=True,
                                            random_state=23))]),
             param_grid={'SVM__C': array([1.00000000e-03, 5.56444444e-01, 1.11188889e+00, 1.66733333e+00,
       2.22277778e+00, 2.77822222e+00, 3.33366667e+00, 3.88911111e+00,
       4.44455556e+00, 5.00000000e+00]),
                         'SVM__gamma': [0.001, 0.5564444444444444,
                                        1.1118888888888887, 1.6673333333333331,
                                        2.2227777777777775, 2.778222222222222,
                                        3.3336666666666663, 3.8891111111111107,
                                        4.444555555555556, 5.0, 'scale',
                                        'auto']},
             scoring='neg_log_loss', verbose=3)

In [41]:
print(gcv.best_params_)
print(gcv.best_score_)

{'SVM__C': 0.5564444444444444, 'SVM__gamma': 'scale'}
-1.0354126932900787


In [42]:
best_model=gcv.best_estimator_
y_pred=best_model.predict(test_df)
y_pred

array([5, 6, 6, ..., 5, 5, 5])

In [43]:
sample_submission_df=pd.read_csv('sample_submission.csv',index_col=0)

In [44]:
sample_submission_df['quality']=y_pred

In [45]:
sample_submission_df.to_csv('sample_submission.csv')

In [56]:
# Leaderboard Score
# Score: 0.45141
# Public score: 0.50007

## 4. MinMax scaling + Radial

In [48]:
scaler=MinMaxScaler()
svm=SVC(kernel='rbf',probability=True,random_state=23)
pipe=Pipeline([('SCL',scaler),('SVM',svm)])
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)

params={'SVM__C':np.linspace(0.001,5,10),'SVM__gamma':list(np.linspace(0.001,5,10))+['scale','auto']}

gcv=GridSearchCV(pipe,param_grid=params,cv=kfold,scoring='neg_log_loss',verbose=3)
gcv.fit(X_train,y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END ...SVM__C=0.001, SVM__gamma=0.001;, score=-1.221 total time=   1.1s
[CV 2/5] END ...SVM__C=0.001, SVM__gamma=0.001;, score=-1.110 total time=   1.1s
[CV 3/5] END ...SVM__C=0.001, SVM__gamma=0.001;, score=-1.156 total time=   1.1s
[CV 4/5] END ...SVM__C=0.001, SVM__gamma=0.001;, score=-1.117 total time=   1.1s
[CV 5/5] END ...SVM__C=0.001, SVM__gamma=0.001;, score=-1.109 total time=   1.2s
[CV 1/5] END SVM__C=0.001, SVM__gamma=0.5564444444444444;, score=-1.118 total time=   1.1s
[CV 2/5] END SVM__C=0.001, SVM__gamma=0.5564444444444444;, score=-1.056 total time=   1.1s
[CV 3/5] END SVM__C=0.001, SVM__gamma=0.5564444444444444;, score=-1.087 total time=   1.2s
[CV 4/5] END SVM__C=0.001, SVM__gamma=0.5564444444444444;, score=-1.072 total time=   1.2s
[CV 5/5] END SVM__C=0.001, SVM__gamma=0.5564444444444444;, score=-1.058 total time=   1.1s
[CV 1/5] END SVM__C=0.001, SVM__gamma=1.1118888888888887;, score=-1.107 total

[CV 5/5] END SVM__C=0.5564444444444444, SVM__gamma=2.778222222222222;, score=-1.025 total time=   1.1s
[CV 1/5] END SVM__C=0.5564444444444444, SVM__gamma=3.3336666666666663;, score=-1.068 total time=   1.1s
[CV 2/5] END SVM__C=0.5564444444444444, SVM__gamma=3.3336666666666663;, score=-1.041 total time=   1.1s
[CV 3/5] END SVM__C=0.5564444444444444, SVM__gamma=3.3336666666666663;, score=-1.037 total time=   1.1s
[CV 4/5] END SVM__C=0.5564444444444444, SVM__gamma=3.3336666666666663;, score=-1.025 total time=   1.1s
[CV 5/5] END SVM__C=0.5564444444444444, SVM__gamma=3.3336666666666663;, score=-1.028 total time=   1.1s
[CV 1/5] END SVM__C=0.5564444444444444, SVM__gamma=3.8891111111111107;, score=-1.068 total time=   1.1s
[CV 2/5] END SVM__C=0.5564444444444444, SVM__gamma=3.8891111111111107;, score=-1.043 total time=   1.1s
[CV 3/5] END SVM__C=0.5564444444444444, SVM__gamma=3.8891111111111107;, score=-1.037 total time=   1.1s
[CV 4/5] END SVM__C=0.5564444444444444, SVM__gamma=3.889111111111

[CV 3/5] END SVM__C=1.1118888888888887, SVM__gamma=scale;, score=-1.033 total time=   1.2s
[CV 4/5] END SVM__C=1.1118888888888887, SVM__gamma=scale;, score=-1.023 total time=   1.2s
[CV 5/5] END SVM__C=1.1118888888888887, SVM__gamma=scale;, score=-1.022 total time=   1.2s
[CV 1/5] END SVM__C=1.1118888888888887, SVM__gamma=auto;, score=-1.075 total time=   1.1s
[CV 2/5] END SVM__C=1.1118888888888887, SVM__gamma=auto;, score=-1.021 total time=   1.2s
[CV 3/5] END SVM__C=1.1118888888888887, SVM__gamma=auto;, score=-1.069 total time=   1.1s
[CV 4/5] END SVM__C=1.1118888888888887, SVM__gamma=auto;, score=-1.050 total time=   1.2s
[CV 5/5] END SVM__C=1.1118888888888887, SVM__gamma=auto;, score=-1.049 total time=   1.2s
[CV 1/5] END SVM__C=1.6673333333333331, SVM__gamma=0.001;, score=-1.122 total time=   1.2s
[CV 2/5] END SVM__C=1.6673333333333331, SVM__gamma=0.001;, score=-1.049 total time=   1.1s
[CV 3/5] END SVM__C=1.6673333333333331, SVM__gamma=0.001;, score=-1.090 total time=   1.1s
[CV 

[CV 2/5] END SVM__C=2.2227777777777775, SVM__gamma=1.6673333333333331;, score=-1.032 total time=   1.2s
[CV 3/5] END SVM__C=2.2227777777777775, SVM__gamma=1.6673333333333331;, score=-1.034 total time=   1.2s
[CV 4/5] END SVM__C=2.2227777777777775, SVM__gamma=1.6673333333333331;, score=-1.023 total time=   1.2s
[CV 5/5] END SVM__C=2.2227777777777775, SVM__gamma=1.6673333333333331;, score=-1.019 total time=   1.2s
[CV 1/5] END SVM__C=2.2227777777777775, SVM__gamma=2.2227777777777775;, score=-1.066 total time=   1.1s
[CV 2/5] END SVM__C=2.2227777777777775, SVM__gamma=2.2227777777777775;, score=-1.039 total time=   1.2s
[CV 3/5] END SVM__C=2.2227777777777775, SVM__gamma=2.2227777777777775;, score=-1.034 total time=   1.3s
[CV 4/5] END SVM__C=2.2227777777777775, SVM__gamma=2.2227777777777775;, score=-1.028 total time=   1.3s
[CV 5/5] END SVM__C=2.2227777777777775, SVM__gamma=2.2227777777777775;, score=-1.027 total time=   1.4s
[CV 1/5] END SVM__C=2.2227777777777775, SVM__gamma=2.77822222222

[CV 4/5] END SVM__C=2.778222222222222, SVM__gamma=3.8891111111111107;, score=-1.035 total time=   1.2s
[CV 5/5] END SVM__C=2.778222222222222, SVM__gamma=3.8891111111111107;, score=-1.052 total time=   1.2s
[CV 1/5] END SVM__C=2.778222222222222, SVM__gamma=4.444555555555556;, score=-1.073 total time=   1.2s
[CV 2/5] END SVM__C=2.778222222222222, SVM__gamma=4.444555555555556;, score=-1.055 total time=   1.2s
[CV 3/5] END SVM__C=2.778222222222222, SVM__gamma=4.444555555555556;, score=-1.046 total time=   1.2s
[CV 4/5] END SVM__C=2.778222222222222, SVM__gamma=4.444555555555556;, score=-1.039 total time=   1.2s
[CV 5/5] END SVM__C=2.778222222222222, SVM__gamma=4.444555555555556;, score=-1.056 total time=   1.2s
[CV 1/5] END SVM__C=2.778222222222222, SVM__gamma=5.0;, score=-1.075 total time=   1.2s
[CV 2/5] END SVM__C=2.778222222222222, SVM__gamma=5.0;, score=-1.058 total time=   1.3s
[CV 3/5] END SVM__C=2.778222222222222, SVM__gamma=5.0;, score=-1.049 total time=   1.2s
[CV 4/5] END SVM__C=

[CV 4/5] END SVM__C=3.8891111111111107, SVM__gamma=0.001;, score=-1.083 total time=   1.2s
[CV 5/5] END SVM__C=3.8891111111111107, SVM__gamma=0.001;, score=-1.070 total time=   1.2s
[CV 1/5] END SVM__C=3.8891111111111107, SVM__gamma=0.5564444444444444;, score=-1.057 total time=   1.2s
[CV 2/5] END SVM__C=3.8891111111111107, SVM__gamma=0.5564444444444444;, score=-1.021 total time=   1.2s
[CV 3/5] END SVM__C=3.8891111111111107, SVM__gamma=0.5564444444444444;, score=-1.043 total time=   1.4s
[CV 4/5] END SVM__C=3.8891111111111107, SVM__gamma=0.5564444444444444;, score=-1.026 total time=   1.3s
[CV 5/5] END SVM__C=3.8891111111111107, SVM__gamma=0.5564444444444444;, score=-1.014 total time=   1.2s
[CV 1/5] END SVM__C=3.8891111111111107, SVM__gamma=1.1118888888888887;, score=-1.063 total time=   1.2s
[CV 2/5] END SVM__C=3.8891111111111107, SVM__gamma=1.1118888888888887;, score=-1.028 total time=   1.2s
[CV 3/5] END SVM__C=3.8891111111111107, SVM__gamma=1.1118888888888887;, score=-1.036 total

[CV 2/5] END SVM__C=4.444555555555556, SVM__gamma=2.778222222222222;, score=-1.050 total time=   1.4s
[CV 3/5] END SVM__C=4.444555555555556, SVM__gamma=2.778222222222222;, score=-1.041 total time=   1.3s
[CV 4/5] END SVM__C=4.444555555555556, SVM__gamma=2.778222222222222;, score=-1.034 total time=   1.2s
[CV 5/5] END SVM__C=4.444555555555556, SVM__gamma=2.778222222222222;, score=-1.046 total time=   1.2s
[CV 1/5] END SVM__C=4.444555555555556, SVM__gamma=3.3336666666666663;, score=-1.075 total time=   1.2s
[CV 2/5] END SVM__C=4.444555555555556, SVM__gamma=3.3336666666666663;, score=-1.055 total time=   1.3s
[CV 3/5] END SVM__C=4.444555555555556, SVM__gamma=3.3336666666666663;, score=-1.045 total time=   1.3s
[CV 4/5] END SVM__C=4.444555555555556, SVM__gamma=3.3336666666666663;, score=-1.039 total time=   1.3s
[CV 5/5] END SVM__C=4.444555555555556, SVM__gamma=3.3336666666666663;, score=-1.054 total time=   1.3s
[CV 1/5] END SVM__C=4.444555555555556, SVM__gamma=3.8891111111111107;, score=

[CV 3/5] END ......SVM__C=5.0, SVM__gamma=auto;, score=-1.060 total time=   1.2s
[CV 4/5] END ......SVM__C=5.0, SVM__gamma=auto;, score=-1.042 total time=   1.2s
[CV 5/5] END ......SVM__C=5.0, SVM__gamma=auto;, score=-1.040 total time=   1.3s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=Pipeline(steps=[('SCL', MinMaxScaler()),
                                       ('SVM',
                                        SVC(probability=True,
                                            random_state=23))]),
             param_grid={'SVM__C': array([1.00000000e-03, 5.56444444e-01, 1.11188889e+00, 1.66733333e+00,
       2.22277778e+00, 2.77822222e+00, 3.33366667e+00, 3.88911111e+00,
       4.44455556e+00, 5.00000000e+00]),
                         'SVM__gamma': [0.001, 0.5564444444444444,
                                        1.1118888888888887, 1.6673333333333331,
                                        2.2227777777777775, 2.778222222222222,
                                        3.3336666666666663, 3.8891111111111107,
                                        4.444555555555556, 5.0, 'scale',
                                        'auto']},
             scoring='neg_log_loss', verbose=3)

In [49]:
print(gcv.best_params_)
print(gcv.best_score_)

{'SVM__C': 5.0, 'SVM__gamma': 0.5564444444444444}
-1.031289737935794


In [51]:
best_model=gcv.best_estimator_

In [52]:
y_pred=best_model.predict(test_df)
y_pred

array([5, 6, 5, ..., 5, 5, 5])

In [53]:
sample_submission_df=pd.read_csv('sample_submission.csv',index_col=0)

In [54]:
sample_submission_df['quality']=y_pred

In [57]:
sample_submission_df.to_csv('sample_submission.csv')

In [58]:
# Leaderboard Score
# Score: 0.45141
# Public score: 0.50007